In [1]:
import datetime as dt
import pandas as pd

In [2]:
# Load the dataset (change file path as needed)
df = pd.read_csv("/Users/gwangyongkim/Downloads/archive (1)/Year 2009-2010.csv",encoding='iso-8859-9')


# Create a copy to work with
data = df.copy()

# Display the first few rows
print(data.head())

# Shape of the dataset
print(f"Dataset contains {data.shape[0]} rows and {data.shape[1]} columns.")

# Check for missing values
print(data.isnull().sum())

  Invoice StockCode                          Description  Quantity   
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12  \
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

      InvoiceDate  Price  Customer ID         Country  
0  12/1/2009 7:45   6.95      13085.0  United Kingdom  
1  12/1/2009 7:45   6.75      13085.0  United Kingdom  
2  12/1/2009 7:45   6.75      13085.0  United Kingdom  
3  12/1/2009 7:45   2.10      13085.0  United Kingdom  
4  12/1/2009 7:45   1.25      13085.0  United Kingdom  
Dataset contains 525461 rows and 8 columns.
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [3]:
###############################################################
# 3. Data Preparation
###############################################################

# Remove rows with missing Customer IDs
data.dropna(subset=['Customer ID'], inplace=True)

# Remove cancelled transactions (Invoice starts with 'C')
data = data[~data["Invoice"].astype(str).str.contains("C", na=False)]

# Only keep transactions with positive quantities
data = data[data["Quantity"] > 0]

# Create a new column for Total Price
data["TotalPrice"] = data["Quantity"] * data["Price"]

# Ensure 'InvoiceDate' is in datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [4]:
###############################################################
# 4. Calculate RFM Metrics
###############################################################

# Define the analysis date (set to a date after the last transaction)
analysis_date = dt.datetime(2010, 12, 11)

# Calculate Recency, Frequency, and Monetary metrics
rfm = data.groupby('Customer ID').agg({
    'InvoiceDate': lambda x: (analysis_date - x.max()).days,  # Recency
    'Invoice': 'nunique',                                    # Frequency
    'TotalPrice': 'sum'                                      # Monetary
})

# Rename columns for clarity
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Filter out customers with zero or negative monetary value
rfm = rfm[rfm['Monetary'] > 0]

# Display summary statistics
print(rfm.describe())

           Recency    Frequency       Monetary
count  4312.000000  4312.000000    4312.000000
mean     91.172542     4.455705    2048.238236
std      96.861457     8.170213    8914.481280
min       1.000000     1.000000       2.950000
25%      18.000000     1.000000     307.987500
50%      53.000000     2.000000     706.020000
75%     136.000000     5.000000    1723.142500
max     374.000000   205.000000  349164.350000


In [5]:
rfm.head()

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,165,11,372.86
12347.0,3,2,1323.32
12348.0,74,1,222.16
12349.0,43,3,2671.14
12351.0,11,1,300.93


In [6]:
###############################################################
# 5. Calculate RFM Scores
###############################################################

# Assign scores based on quantiles
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])

# Combine Recency and Frequency scores into a single RFM Score
rfm["RFMScore"] = rfm["RecencyScore"].astype(str) + rfm["FrequencyScore"].astype(str)

In [7]:
###############################################################
# 6. Segment Customers
###############################################################

# Define RFM segment mapping
segment_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At Risk',
    r'[1-2]5': 'Cannot Lose',
    r'3[1-2]': 'About to Sleep',
    r'33': 'Need Attention',
    r'[3-4][4-5]': 'Loyal Customers',
    r'41': 'Promising',
    r'51': 'New Customers',
    r'[4-5][2-3]': 'Potential Loyalists',
    r'5[4-5]': 'Champions'
}

# Apply the mapping
rfm['Segment'] = rfm['RFMScore'].replace(segment_map, regex=True)

# Display the segments
print(rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}))

                        Recency  Frequency     Monetary      
                           mean       mean         mean count
Segment                                                      
About to Sleep        53.819242   1.201166   441.320000   343
At Risk              152.158756   3.073650  1188.878316   611
Cannot Lose          124.116883   9.116883  4099.450000    77
Champions              7.119155  12.553544  6852.264167   663
Hibernating          213.885714   1.126108   403.977836  1015
Loyal Customers       36.287062   6.830189  2746.067353   742
Need Attention        53.265700   2.449275  1060.357005   207
New Customers          8.580000   1.000000   386.199200    50
Potential Loyalists   18.793037   2.017408   729.510986   517
Promising             25.747126   1.000000   367.086782    87


In [8]:
###############################################################
# 7. Save Results to CSV
###############################################################

# Save the RFM table to a CSV file
rfm.reset_index(inplace=True)  # Reset index to include Customer ID as a column
rfm.to_csv("rfm_segments.csv", index=False)

In [9]:
###############################################################
# 8. Create a Function for Reusability
###############################################################

def create_rfm(dataframe, analysis_date, save_csv=False):
    """
    Function to create an RFM table with segments.
    
    Parameters:
        dataframe (pd.DataFrame): The transaction data.
        analysis_date (datetime): The date of analysis.
        save_csv (bool): Whether to save the RFM table to a CSV file.
        
    Returns:
        rfm (pd.DataFrame): The RFM table with segments.
    """
    dataframe = dataframe.copy()
    
    # Data preparation
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    dataframe.dropna(subset=['Customer ID'], inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].astype(str).str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    
    # Calculate RFM metrics
    rfm = dataframe.groupby('Customer ID').agg({
        'InvoiceDate': lambda x: (analysis_date - x.max()).days,
        'Invoice': 'nunique',
        'TotalPrice': 'sum'
    })
    rfm.columns = ['Recency', 'Frequency', 'Monetary']
    rfm = rfm[rfm['Monetary'] > 0]
    
    # Calculate RFM scores
    rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])
    rfm["RFMScore"] = rfm["RecencyScore"].astype(str) + rfm["FrequencyScore"].astype(str)
    
    # Map RFM scores to segments
    rfm['Segment'] = rfm['RFMScore'].replace(segment_map, regex=True)
    
    if save_csv:
        rfm.reset_index(inplace=True)
        rfm.to_csv("rfm_segments.csv", index=False)
    
    return rfm

# Example usage
rfm_table = create_rfm(data, analysis_date=dt.datetime(2010, 12, 11), save_csv=True)

## Conclusion

### RFM Segmentation Insights and Action Plan

This analysis categorizes customers into nine distinct segments based on **Recency, Frequency, and Monetary (RFM)** metrics. Below are tailored strategies for each segment and an overall action plan to drive business growth:

1. Champions

- Insights : Most valuable customers with the highest frequency, recency, and spending.
- Action Plan
    - Reward their loyalty with exclusive benefits, early access, or premium offers.
    - Provide upselling opportunities with high-value products or bundles.
    - Involve them in feedback initiatives to improve services and reinforce their connection to the brand.


2. Loyal Customers

- Insights : Regular buyers with strong brand loyalty and high spending, though slightly less recent than Champions.
- Action Plan
    - Strengthen engagement through loyalty programs and personalized campaigns.
    - Offer occasional surprise rewards or discounts to maintain their trust.
    - Encourage them to act as brand advocates through referral incentives.

3. At Risk

- Insights : Previously frequent, high-spending customers who have not interacted recently.
- Action Plan
    - Deploy win-back campaigns with personalized, time-sensitive offers.
    - Use targeted communication to remind them of their past interactions and loyalty points.
    - Address potential concerns or dissatisfaction through surveys or direct communication.

4. Hibernating

- Insights : The largest segment with poor recency, low frequency, and minimal spending.
- Action Plan
    - Run cost-effective reactivation campaigns to re-engage this segment.
    - Investigate potential causes of disengagement, such as market competition or product fit.
    - Focus on revitalizing only customers showing interest or responsiveness to marketing efforts.

5. Potential Loyalists

- Insights : Customers with recent interactions and moderate spending, indicating growth potential.
- Action Plan
    - Encourage repeat purchases with tailored promotions and recommendations.
    - Enhance their experience through excellent customer service and engagement.
    - Monitor their behavior closely to transition them into Loyal Customers.

6. Need Attention

- Insights : Moderate engagement but at risk of losing interest due to low frequency.
- Action Plan
    - Reignite their interest with personalized communication and attractive offers.
    - Showcase products or services related to their past purchases to stimulate repeat sales.
    - Create a sense of urgency with limited-time discounts or bundled deals.

7. New Customers

- Insights : First-time buyers still exploring the brand.
- Action Plan
     - Welcome them warmly with educational content and onboarding campaigns
     - Offer discounts or free trials to encourage a second purchase.
     - Gather feedback on their first experience to improve future customer journeys.

8. Promising

- Insights : Customers with moderate recency and low frequency who show potential for higher engagement.
- Action Plan
    - Use data-driven recommendations to cross-sell or upsell relevant products.
    - Provide incentives for repeat purchases, such as loyalty points or free shipping.
    - Test engagement through personalized campaigns targeting their specific needs.

9. Cannot Lose

- Insights : High-value, frequent buyers who are at significant risk of churn.
- Action Plan
    - Prioritize retention efforts with direct, personalized offers.
    - Deliver exclusive VIP benefits or recognition programs to restore their interest.
    - Act quickly to identify and address the reasons for disengagement.

### General Recommendatoins

1. Prioritize High-Value Segments (Champions, Loyal Customers, Cannot Lose):
    - Focus investments on retaining these groups, as they drive the majority of revenue.

2. Target Growth Segments (Potential Loyalists, Promising):
    - Nurture these segments to develop them into Loyal Customers or Champions through personalized campaigns.

3. Re-engage At Risk and Hibernating Customers:
    - Allocate resources carefully, focusing on those showing responsiveness or potential.

4. Utilize Data Analytics:
    - Continuously track RFM metrics to evaluate campaign effectiveness and adapt strategies accordingly.

5. Foster a Feedback Loop:
    - Collect insights from Champions and Loyal Customers to enhance products and customer experiences, ensuring long-term loyalty.

By aligning strategies with the unique needs of each segment, the company can maximize customer lifetime value, boost engagement, and foster sustainable growth.